In [1]:
import torch
import torch.nn as nn
import numpy as np

import os

In [2]:
class Arguments:
    res_path = './results'
    model_path = './models'
    epoch  = 20
    patience = -1
    teacher = True
    bidi = False
    test = False
    shrd_dec_emb = False 
    btstrp  = None 
    lm = False 
    toy = False 
    pretty = False 
    mmi = False
    seq2seq = False 
    drp = 0.3
    num_lyr = 1
    lr = 0.0001
    bt_siz = 64
    beam = 1
    vocab_size = 50005 
    response_vocab_size = 32149
    emb_size = 300
    hid_size = 128
    ut_hid_size = 128
    ses_hid_size = 256
    dec_hid_size = 256
    kb_embed_size = 100
    kb_size = 9274341
    kb_relations_size= 571
    use_embed = True




In [3]:
options = Arguments()

In [4]:

ent_embed = np.load('transe_dir/ent_embed.pkl.npy')
rel_embed = np.load('transe_dir/rel_embed.pkl.npy')

new_row = np.zeros((1,100), dtype=np.float32)
    
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <pad_kb>
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <nkb>

rel_embed = np.vstack([new_row, rel_embed]) # corr. to <pad_kb>
rel_embed = np.vstack([new_row, rel_embed]) 



In [5]:
def mini_batch_processing(mini_batch):
    mem_size = len(mini_batch[-1][6].split('|'))
    if mem_size > 10000 :
        mem_size =  10000
    u1_text = np.array([s[0][0] for s in mini_batch])
    u1_kb   = np.array([s[1][0] for s in mini_batch])
    u2_text = np.array([s[0][1] for s in mini_batch] )
    u2_kb   = np.array([s[1][1] for s in mini_batch])
    target_kb = np.array([s[2] for s in mini_batch])
    target_utterance = np.array([s[3] for s in mini_batch])
    cand_ent =  [ [int(i) for i in s[6].split('|')] for s in mini_batch]
    cand_ent =  [ s[:-1][:mem_size-1]+[1]  for s in cand_ent]
    cand_ent =  np.array([ s + [0]*(mem_size-len(s)) for s in cand_ent])
    
    cand_rel =  [ [int(i) for i in s[7].split('|')] for s in mini_batch]
    cand_rel =  [ s[:-1][:mem_size-1]+[1]  for s in cand_rel]
    cand_rel =  np.array([ s + [0]*(mem_size - len(s)) for s in cand_rel])
    
    cand_val =  [ [int(i) for i in s[8].split('|')] for s in mini_batch]
    cand_val =  [ s[:-1][:mem_size-1]+[1]  for s in cand_val]
    cand_val =  np.array([s + [0]*(mem_size-len(s)) for s in cand_val])
    '''
    mem_weights = ((cand_val!=0) * (cand_val!=1) ).astype(int)
    
   
    labels = []
    candidates = []
    for i in range(64):
        positive_samples_i =list( set(target_kb[i])-{0})
        labels_i = [1]*len(positive_samples_i)
        negative_samples_i = list(set(cand_val[i])-(set(target_kb[i])-{0}) )
        if len(negative_samples_i) > 0:
            negative_samples_subset_i  = list( np.random.choice(negative_samples_i,20 - len(labels_i)) )
        else :
            negative_samples_subset_i =[]

        labels_i = labels_i + [0] *(20 - len(labels_i))
        candidates_i = positive_samples_i + negative_samples_subset_i
        candidates_i = candidates_i + (20-len(candidates_i)) *[0]

        labels.append(labels_i)
        candidates.append( candidates_i )
    
    c = np.array(labels)
    candidates_pred = np.array(candidates)
    
    '''
    
    b , l = cand_val.shape
    c = np.zeros((b, l))
    for i in range(b):
        for j in range(l):
            c[i,j] = int( (cand_val[i,j] in target_kb[i]) and cand_val[i,j] != 0) 
    
    
    
    return(u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,  cand_ent, cand_rel, cand_val,c)


In [6]:
loss_dec =  nn.CrossEntropyLoss()
softmax = nn.LogSoftmax(dim = 1)

In [7]:
from model import  *

In [8]:
import os

In [9]:
def padded_candidates(t):
    n_samples = len(t)
    
    for i in range(n_samples):
        cand = t[i][6].split('|')
        n_cand = len(cand)
        if n_cand  < 10:
            t[i][6] = '|'.join( cand + ['0']*(10-n_cand))
            cand = t[i][7].split('|')
            t[i][7] = '|'.join( cand + ['0']*(10-n_cand))
            cand = t[i][8].split('|')
            t[i][8] = '|'.join( cand + ['0']*(10-n_cand))
    return(t)

In [10]:
training_files=["dump/"+x for x in os.listdir("/home/tony/QA/dump") if x.startswith('train_')]




In [11]:
import pickle as pkl

In [12]:
model_ = model(options)

model_ = nn.DataParallel(model_).cuda()

/home/tony/miniconda2/lib/python2.7/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [13]:
from torch import optim
from tqdm import tqdm_notebook

In [14]:
param_kg_loss = []
param_dec_loss = []
for t in model_.named_parameters():
    if 'kg_predict' in t[0]  :
        param_kg_loss.append(t[1])
    else:
        param_dec_loss.append(t[1])

In [15]:
optimizer_kg_loss = optim.Adam( param_kg_loss, options.lr)

optimizer_dec_loss = optim.Adam( param_dec_loss, options.lr)


In [16]:
file = training_files[0]
t = pkl.load(open(file, 'rb'))
t = padded_candidates(t)
np.random.shuffle(t)
n_batches = (len(t)//64)
t = sorted(t, key = lambda x : -1 *len(x[6].split('|')))


In [17]:
total_loss = 0
total_kg_loss = 0
total_dec_loss = 0

In [19]:

for e in range(20):
    total_loss = 0
    total_kg_loss = 0
    total_dec_loss = 0

    
    for file in tqdm_notebook(training_files):
        t = pkl.load(open(file, 'rb'))
        t = padded_candidates(t)
        np.random.shuffle(t)
        n_batches = (len(t)//128)
        t = sorted(t, key = lambda x : -1 *len(x[6].split('|')))
        
        for i in tqdm_notebook(range(1, n_batches-1)):
            u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,\
                                cand_ent, cand_rel, cand_val, c = mini_batch_processing(t[ (i-1) * 128 : i *128])

            u1_text = np.flip(u1_text, axis = 1).copy()
            u1_kb = np.flip(u1_kb, axis = 1).copy()

            u2_text = np.flip(u2_text, axis = 1).copy()
            u2_kb = np.flip(u2_kb, axis = 1).copy()

            u1_text = torch.LongTensor(u1_text).cuda()
            u2_text = torch.LongTensor(u2_text).cuda()

            u1_kb = torch.FloatTensor(ent_embed[u1_kb]).cuda()
            u2_kb = torch.FloatTensor(ent_embed[u2_kb]).cuda()

            target_utterance = torch.LongTensor(target_utterance).cuda()

            cand_ent_embed  = torch.FloatTensor(ent_embed[cand_ent]).cuda()
            cand_rel_embed  = torch.FloatTensor(rel_embed[cand_rel]).cuda()
            cand_val_embed  = torch.FloatTensor(ent_embed[cand_val]).cuda()
            cand_pred_embed = torch.FloatTensor(ent_embed[cand_val]).cuda()

            for stage in range(2):
                pred = model_(u1_text, u1_kb, u2_text, u2_kb,target_utterance,\
                            cand_ent_embed, cand_rel_embed, cand_val_embed)

                pred_text = pred[0]
                pred_kg = pred[1]
                if stage == 0 :
                    pred_text = pred_text[:, :-1, :].contiguous().view(-1, pred_text.size(2))
                    u3 = target_utterance[:, 1:].contiguous().view(-1)

                    dec_loss = loss_dec(pred_text, u3)

                    optimizer_dec_loss.zero_grad()

                    dec_loss.backward()
                    torch.nn.utils.clip_grad_norm_(param_dec_loss, 0.1)
                    optimizer_dec_loss.step()
                else:
                    c = torch.FloatTensor(c)/(0.000001 + torch.FloatTensor(c).sum(dim = 1).unsqueeze(1)  )
                    c = c.cuda()
                    kg_loss = -1 * (softmax(pred_kg) * c).mean() 

                    optimizer_kg_loss.zero_grad()
                    kg_loss.backward()
                    torch.nn.utils.clip_grad_norm_(param_kg_loss, 0.1)
                    optimizer_kg_loss.step()

            loss = kg_loss.item() + dec_loss.item()

            total_loss = total_loss + loss
            total_kg_loss = total_kg_loss + kg_loss.item()
            total_dec_loss = total_dec_loss + dec_loss.item()
        #print(q.var(dim=0).min().item(), q.var(dim=0).max().item()  )
        print("loss in epoch :", e, "loss :", total_loss, "loss decoder: ", \
              total_dec_loss, "loss kg: ", total_kg_loss)



Utterance_encoder.py:51: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  gru_out, gru_hid = self.rnn( emb, h_0)
InterUtterance_encoder.py:23: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  h_o, h_n = self.rnn(x, h_0)
Decoder.py:66: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  hid_o, hid_n = self.rnn(target_emb, init_hidn)


('loss in epoch :', 0, 'loss :', 564.7704111919738, 'loss decoder: ', 549.1805108785629, 'loss kg: ', 15.589900313410908)


('loss in epoch :', 0, 'loss :', 746.9072390885558, 'loss decoder: ', 716.7643297016621, 'loss kg: ', 30.142909386893734)


('loss in epoch :', 0, 'loss :', 905.7425081342808, 'loss decoder: ', 862.9586979448795, 'loss kg: ', 42.783810189401265)


('loss in epoch :', 0, 'loss :', 1055.8634137352346, 'loss decoder: ', 1000.6647595763206, 'loss kg: ', 55.198654158913996)


...

('loss in epoch :', 19, 'loss :', 1406.5481054204574, 'loss decoder: ', 1159.5395977795124, 'loss kg: ', 247.00850764094503)


('loss in epoch :', 19, 'loss :', 1459.6506862191309, 'loss decoder: ', 1203.2867176309228, 'loss kg: ', 256.3639685882081)


('loss in epoch :', 19, 'loss :', 1508.219023229205, 'loss decoder: ', 1243.5201616808772, 'loss kg: ', 264.69886154832784)


('loss in epoch :', 19, 'loss :', 1559.2209719535604, 'loss decoder: ', 1285.538764089346, 'loss kg: ', 273.68220786421443)


('loss in epoch :', 19, 'loss :', 1610.2091900502564, 'loss decoder: ', 1327.3763880506158, 'loss kg: ', 282.83280199964065)



In [171]:
torch.save( model_.module.state_dict(),'train.pt')